In [3]:
!pip install pulp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 86.9 MB/s eta 0:00:00


In [4]:
import ipywidgets as widgets
import numpy as np
import matplotlib.pyplot as plt
from pulp import *

In [7]:
# Proporcje składników w batonikach
MARS = {"Czekolada": 0.5, "Caramel": 0.2, "Nuts": 0, "Oil": 0.05, "Sugar": 0.3}
SNICKERS = {"Czekolada": 0.4, "Caramel": 0.2, "Nuts": 0.2, "Oil": 0.05, "Sugar": 0.3}

# Slider do wyboru parametrów
slider_style = {"description_width": "initial"}

mars_price_slider = widgets.IntSlider(min=1, max=400, value=100, description="Mars Price Per KG", style=slider_style)
snickers_price_slider = widgets.IntSlider(min=1, max=400, value=105, description="Snickers Price Per KG", style=slider_style)
cocoa_price_slider = widgets.IntSlider(min=1, max=100, value=20, description="Cocoa Cost Per KG", style=slider_style)
caramel_price_slider = widgets.IntSlider(min=1, max=100, value=15, description="Caramel Cost Per KG", style=slider_style)
oil_price_slider = widgets.IntSlider(min=1, max=100, value=7, description="Oil Cost Per KG", style=slider_style)
sugar_price_slider = widgets.IntSlider(min=1, max=100, value=20, description="Sugar Cost Per KG", style=slider_style)
nuts_price_slider = widgets.IntSlider(min=1, max=100, value=50, description="Nuts Cost Per KG", style=slider_style)
max_packages_slider = widgets.IntSlider(min=1, max=20000, value=10000, description="Max Packages Amount", style=slider_style)
max_nuts_slider = widgets.IntSlider(min=1, max=20000, value=3000, description="Max Nuts Amount", style=slider_style)
max_cocoa_slider = widgets.IntSlider(min=1, max=20000, value=6000, description="Max Cocoa Amount", style=slider_style)
bar_proportion_slider = widgets.IntSlider(min=0, max=100, value=50, description="Mars-Snickers Proportion", style=slider_style)


def bar_production(
    mars_price_per_kg: int = 100,
    snickers_price_per_kg: int = 105,
    cocoa_price_per_kg: int = 20,
    caramel_price_per_kg: int = 15,
    oil_price_per_kg: int = 7,
    sugar_price_per_kg: int = 20,
    nuts_price_per_kg: int = 50,
    max_packages: int = 10000,
    max_nuts: int = 3000,
    max_cocoa: int = 6000,
    bar_proportions: int = 50,
    printable=True,
):
    prob = LpProblem("Candy Production Optimization", LpMaximize)

    # Zmienna dla ilości Mars i Snickers
    snickers = LpVariable("SNICKERS", lowBound=0)
    mars = LpVariable("MARS", lowBound=0)

    # Zmienna dla składników
    cocoa = LpVariable("Cocoa", lowBound=0, upBound=max_cocoa)
    caramel = LpVariable("Caramel", lowBound=0)
    nuts = LpVariable("Nuts", lowBound=0, upBound=max_nuts)
    oil = LpVariable("Oil", lowBound=0)
    sugar = LpVariable("Sugar", lowBound=0)

    # Dostępne zasoby
    prob += cocoa <= max_cocoa, "MaxCocoa"
    prob += nuts <= max_nuts, "MaxNuts"
    prob += mars * MARS["Czekolada"] + snickers * SNICKERS["Czekolada"] == cocoa, "Cocoa Requirement"
    prob += snickers * SNICKERS["Nuts"] == nuts, "Nuts Requirement"
    prob += mars * MARS["Caramel"] + snickers * SNICKERS["Caramel"] == caramel, "Caramel Requirement"
    prob += mars * MARS["Oil"] + snickers * SNICKERS["Oil"] == oil, "Oil Requirement"
    prob += mars * MARS["Sugar"] + snickers * SNICKERS["Sugar"] == sugar, "Sugar Requirement"

    # Ograniczenie liczby opakowań
    prob += (mars * (100 - bar_proportions) / 100) + (snickers * bar_proportions / 100) <= max_packages, "MaxPackages"

    # Maksymalizacja zysku
    prob += mars_price_per_kg * mars + snickers_price_per_kg * snickers - (cocoa_price_per_kg * cocoa + caramel_price_per_kg * caramel + oil_price_per_kg * oil + sugar_price_per_kg * sugar + nuts_price_per_kg * nuts), "Profit"


    # Rozwiązanie problemu
    prob.solve()

    if printable:
        print("Status:", LpStatus[prob.status])
        for v in prob.variables():
            print(v.name, "=", v.varValue)
        print(f"Total Profit: {value(prob.objective)} PLN")

    # Rysowanie wykresu
    ingredients = ['Czekolada', 'Cukier', 'Olej', 'Karmel', 'Orzechy']
    values = [
        cocoa.varValue, sugar.varValue, oil.varValue, caramel.varValue, nuts.varValue
    ]

    plt.figure(figsize=(10, 6))
    plt.bar(ingredients, values, color='lightblue')
    plt.title("Required Ingredients for Production")
    plt.ylabel('Amount (kg)')
    plt.tight_layout()
    plt.show()



In [8]:
widgets.interact(
    bar_production,
    mars_price_per_kg=mars_price_slider,
    snickers_price_per_kg=snickers_price_slider,
    cocoa_price_per_kg=cocoa_price_slider,
    caramel_price_per_kg=caramel_price_slider,
    oil_price_per_kg=oil_price_slider,
    sugar_price_per_kg=sugar_price_slider,
    nuts_price_per_kg=nuts_price_slider,
    max_packages=max_packages_slider,
    max_nuts=max_nuts_slider,
    max_cocoa=max_cocoa_slider,
    bar_proportions=bar_proportion_slider,
)


interactive(children=(IntSlider(value=100, description='Mars Price Per KG', max=400, min=1, style=SliderStyle(…

<function __main__.bar_production(mars_price_per_kg: int = 100, snickers_price_per_kg: int = 105, cocoa_price_per_kg: int = 20, caramel_price_per_kg: int = 15, oil_price_per_kg: int = 7, sugar_price_per_kg: int = 20, nuts_price_per_kg: int = 50, max_packages: int = 10000, max_nuts: int = 3000, max_cocoa: int = 6000, bar_proportions: int = 50, printable=True)>